# 01 - Data and Data import

In this first section we will detail everything about the dataset and the pre-preparation we had to do to it in order to allow us to use it for our project. The [data](https://www.simpleweb.org/wiki/index.php/SSH_datasets) that we used was a selection of SSH logs from the linked web page. Originally we wanted to attempt to access the data directly from the [source link](http://traces.simpleweb.org/ssh_datasets/dataset2_log_files.tgz), however this proved problematic due to the nature of the file.

We downloaded and inspected the data to find that it was off the following structure:

* `dataset2_log_files.tgz`
    * `dataset2_log_files.tar`
        * `ip-address1`
            * `auth.log.anon`
        * `ip-address2`
            * `auth.log.anon`
        *      ...
        * `ip-address73`
            * `auth.log.anon`
            
Essentially what we had were 73 seperate `.log` files for different ips on the network that the data was recorded on (Campus network of the University of Twente, Netherlands). In order to work with this in the way that we wanted we had to do rather a lot of manual file management to allow us to use this data.

As part of this, we read the details on the page that we downloaded the data from, which had this to say:
>     1. Merging: On some machines, the authentication logs were distributed over hostname.messages and hostname.warn. We have merged those log files, sorted them again (if necessary), and removed any introduced duplicates.
    2. Renaming: The file names have been changed from hostname.extension into anonymized_IP_address.extension. As such, the log files can easily be correlated with the flow data.
    3. Anonymization: We have replaced any usernames by "XXXXX" and hostnames by the anonymized IP address of the considered host.
    
Due to the anonymising of the data the ip address we see are not the originals, but every instance of them corresponds to the same value. Another key point that we will touch on in the next section of the report is how the usernames have been anonymised to "XXXXX". This anonymising process has caused the log files to be represented as `auth.log.anon` and so for ease of reading and accessibility while keeping potentially important data intact we had to do the following steps:
* Copy each `auth.log.anon` file in turn to the GitHub repository on my hardrive in order to push to the master.
    * As part of this process as we copied each in turn from their respective ip-address folder we renamed the file to "1.log" for the first file, "2.log" for the second file etc.
    * We've changed the file extension and this allows us to read the log files in our text editor or a program such as excel as normal.
* "26.log" was especially large and this highlighted that it would be beneficial to compress with gzip our log files, so we did this for all 73 files.
* In order to keep track of the ip addresses so that we know where each log has come from, we made a text file, seperated by commas, detailing each ip address in turn (so the 1st ip address corresponds to it's appropriate .log file.

In the code that follows you'll see how we then handled this data in order to create a good dataset to work on.

## Setup

First we start by importing the packages we require for this document.

In [1]:
import pandas as pd
import requests
import tarfile
import zipfile
from urllib.request import urlopen
from io import StringIO
import datetime as dt

In this next step we open our list of ips mentioned above read them into the variable `ip`. We check the length and see that the list is 73 items long (as expected) and that the first ips are correct when cross referenced with our original data.

In [2]:
global ip
text_file = urlopen("https://github.com/Galeforse/DST-Assessment-03/raw/master/Data/anon_ips_list2.txt")
ip = text_file.read().decode('utf-8').split(',')
print(len(ip))
ip[0:5]

73


['161.166.1.22',
 '161.166.1.23',
 '161.166.1.27',
 '161.166.1.38',
 '161.166.1.54']

In order to stream line the import process we defined the following function which does several things in one.

In [3]:
def log_import(x):
    global ip
    y = str(x)
    root = "https://github.com/Galeforse/DST-Assessment-03/raw/master/Data/log_files/"
    df1 = pd.read_csv(root + y +".log.gz",names=["log"])   
    df1.insert(0, "anon_ip",ip[x-1], allow_duplicates = True)
    return(df1)

The function takes one variable as an input and this corresponds to the log number that we wish to import. Then using string manipulation we read in the log file with 

```pd.read_csv(root + y + ".log.gz",names=["log"]```

This function combines the root of the url (the folder the file is found in) the number of the log file (gathered from the function's one variable) and then adds the file extension. It then adds a column detailing the corresponding anonymised ip from the list of ips we imported in the previous step.

Later on in this report we became more familiar with using RegEx in python and we may have been able to streamline this entire workbook to a much smaller file by matching files and combining in a single function, but at the time we hadn't quite got to grips with using regular expressions, and the above function worked well for what we needed to do.

We quickly test the function on our first log file, noticing that the `anon_ip` is the same for all values as expected.

In [4]:
log_import(1)

,anon_ip,log
0,161.166.1.22,Jan 5 08:13:51 161.166.1.22 sshd[42590]: pam_...
1,161.166.1.22,Jan 5 08:13:53 161.166.1.22 sshd[42590]: Fail...
2,161.166.1.22,Jan 5 08:13:55 161.166.1.22 sshd[42590]: Fail...
3,161.166.1.22,Jan 5 08:13:57 161.166.1.22 sshd[42590]: Fail...
4,161.166.1.22,Jan 5 08:14:00 161.166.1.22 sshd[42590]: Fail...
...,...,...
113063,161.166.1.22,Feb 28 23:01:15 161.166.1.22 sshd[52192]: Rece...
113064,161.166.1.22,Feb 28 23:01:15 161.166.1.22 sshd[52192]: pam_...
113065,161.166.1.22,Feb 28 23:01:15 161.166.1.22 sshd[52192]: pam_...
113066,161.166.1.22,Feb 28 23:01:15 161.166.1.22 sshd[52218]: Acce...


## Importing Log Files

We couldn't find a way to automate the next steps any better and so imported each of the 73 log files to it's own dataframe in turn. (conducted in blocks of 10 in case of errors arising)

In [5]:
start = dt.datetime.now()
df1 = log_import(1)
df2 = log_import(2)
df3 = log_import(3)
df4 = log_import(4)
df5 = log_import(5)
df6 = log_import(6)
df7 = log_import(7)
df8 = log_import(8)
df9 = log_import(9)
print("Imported data in: "+str(dt.datetime.now()-start))

Imported data in: 0:00:08.681687


In [6]:
start = dt.datetime.now()
df10 = log_import(10)
df11 = log_import(11)
df12 = log_import(12)
df13 = log_import(13)
df14 = log_import(14)
df15 = log_import(15)
df16 = log_import(16)
df17 = log_import(17)
df18 = log_import(18)
df19 = log_import(19)
print("Imported data in: "+str(dt.datetime.now()-start))

Imported data in: 0:00:09.559229


In [7]:
start = dt.datetime.now()
df20 = log_import(20)
df21 = log_import(21)
df22 = log_import(22)
df23 = log_import(23)
df24 = log_import(24)
df25 = log_import(25)
df26 = log_import(26)
df27 = log_import(27)
df28 = log_import(28)
df29 = log_import(29)
print("Imported data in : "+str(dt.datetime.now()-start))

Imported data in : 0:00:26.687143


In [8]:
start = dt.datetime.now()
df30 = log_import(30)
df31 = log_import(31)
df32 = log_import(32)
df33 = log_import(33)
df34 = log_import(34)
df35 = log_import(35)
df36 = log_import(36)
df37 = log_import(37)
df38 = log_import(38)
df39 = log_import(39)
print("Imported data in: "+str(dt.datetime.now()-start))

Imported data in: 0:00:06.506241


In [9]:
start = dt.datetime.now()
df40 = log_import(40)
df41 = log_import(41)
df42 = log_import(42)
df43 = log_import(43)
df44 = log_import(44)
df45 = log_import(45)
df46 = log_import(46)
df47 = log_import(47)
df48 = log_import(48)
df49 = log_import(49)
print("Imported data in: "+str(dt.datetime.now()-start))

Imported data in: 0:00:09.440936


In [10]:
start = dt.datetime.now()
df50 = log_import(50)
df51 = log_import(51)
df52 = log_import(52)
df53 = log_import(53)
df54 = log_import(54)
df55 = log_import(55)
df56 = log_import(56)
df57 = log_import(57)
df58 = log_import(58)
df59 = log_import(59)
print("Imported data in: "+str(dt.datetime.now()-start))

Imported data in: 0:00:11.738392


In [11]:
start = dt.datetime.now()
df60 = log_import(60)
df61 = log_import(61)
df62 = log_import(62)
df63 = log_import(63)
df64 = log_import(64)
df65 = log_import(65)
df66 = log_import(66)
df67 = log_import(67)
df68 = log_import(68)
df69 = log_import(69)
df70 = log_import(70)
df71 = log_import(71)
df72 = log_import(72)
df73 = log_import(73)
print("Imported data in: "+str(dt.datetime.now()-start))

Imported data in: 0:00:14.427739


For our penultimate step in setting up this data we use the pandas `concat` function to combine all of our log files into one, one after another. We then reset the index of the file and drop the old indexs (which become a new column when the index is reset)

In [12]:
dftest = pd.concat([df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df21,df22,df23,df24,df25,df26,df27,df28,df29,df30,
                       df31,df32,df33,df34,df35,df36,df37,df38,df39,df40,df41,df42,df43,df44,df45,df46,df47,df48,df49,df50,df51,df52,df53,df54,df55,df56,df57,df58,
                       df59,df60,df61,df62,df63,df64,df65,df66,df67,df68,df69,df70,df71,df72,df73])

In [13]:
dftest = dftest.reset_index()
dftest = dftest.drop(columns=["index"],axis=1)
dftest

,anon_ip,log
0,161.166.1.23,Jan 5 03:23:40 161.166.1.23 sshd[27076]: pam_...
1,161.166.1.23,Jan 5 03:23:42 161.166.1.23 sshd[27076]: Fail...
2,161.166.1.23,Jan 5 03:23:44 161.166.1.23 sshd[27076]: Fail...
3,161.166.1.23,Jan 5 03:23:47 161.166.1.23 sshd[27076]: Fail...
4,161.166.1.23,Jan 5 03:23:49 161.166.1.23 sshd[27076]: Fail...
...,...,...
15880517,161.166.236.154,Feb 23 05:20:38 161.166.236.154 sshd[16644]: I...
15880518,161.166.236.154,Feb 23 05:20:41 161.166.236.154 sshd[16662]: I...
15880519,161.166.236.154,Feb 23 05:20:44 161.166.236.154 sshd[16708]: I...
15880520,161.166.236.154,Feb 23 06:29:04 161.166.236.154 sshd[1282]: Us...


## Exporting and compressing for further use

Finally we write our data set to a new log file compressed with .gzip.

In [14]:
start = dt.datetime.now()
dftest.to_csv(path_or_buf="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/Data/master_log.csv.gz",index=False,compression="gzip")
print("Log written to czv.gz in: "+str(dt.datetime.now()-start))

Log written to czv.gz in: 0:02:02.223963


For testing purposes and to deal with memory limitations in some of the next methods in the report we also made several smaller versions of the data set using the sample function. We again reset the index and made sure that the data would appear in the same ip order as the original data set.
### 10 percent data

In [15]:
smp_size = round(len(dftest)/10)
dftest_10 = dftest.sample(n=smp_size,random_state=7)
dftest_10 = dftest_10.sort_index()
dftest_10 = dftest_10.reset_index()
dftest_10 = dftest_10.drop(columns=["index"],axis=1)

In [16]:
start = dt.datetime.now()
dftest_10.to_csv(path_or_buf="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/Data/master_log_10.csv.gz",index=False,compression="gzip")
print("Log written to czv.gz in: "+str(dt.datetime.now()-start))

Log written to czv.gz in: 0:00:13.629114


### 5 percent data

In [17]:
smp_size = round(len(dftest)/20)
dftest_5 = dftest.sample(n=smp_size,random_state=7)
dftest_5 = dftest_5.sort_index()
dftest_5 = dftest_5.reset_index()
dftest_5 = dftest_5.drop(columns=["index"],axis=1)

In [18]:
start = dt.datetime.now()
dftest_5.to_csv(path_or_buf="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/Data/master_log_5.csv.gz",index=False,compression="gzip")
print("Log written to czv.gz in: "+str(dt.datetime.now()-start))

Log written to czv.gz in: 0:00:07.067844


### 50 percent data

In [19]:
smp_size = round(len(dftest)/2)
dftest_50 = dftest.sample(n=smp_size,random_state=7)
dftest_50 = dftest_50.sort_index()
dftest_50 = dftest_50.reset_index()
dftest_50 = dftest_50.drop(columns=["index"],axis=1)

In [20]:
start = dt.datetime.now()
dftest_50.to_csv(path_or_buf="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/Data/master_log_50.csv.gz",index=False,compression="gzip")
print("Log written to czv.gz in: "+str(dt.datetime.now()-start))

Log written to czv.gz in: 0:01:04.750243


### 40 percent data

In [21]:
smp_size = round((len(dftest)*4)/10)
dftest_40 = dftest.sample(n=smp_size,random_state=7)
dftest_40 = dftest_40.sort_index()
dftest_40 = dftest_40.reset_index()
dftest_40 = dftest_40.drop(columns=["index"],axis=1)

In [22]:
start = dt.datetime.now()
dftest_40.to_csv(path_or_buf="G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/Data/master_log_40.csv.gz",index=False,compression="gzip")
print("Log written to czv.gz in: "+str(dt.datetime.now()-start))

Log written to czv.gz in: 0:00:49.837420


Now on GitHubDesktop we just save and push these files to the repository, and are now ready to access them from anywhere, for use in further code.